This notebook controls training. You should define:
1. the number of workers
2. the number of cpus each worker has
3. the corresponding job name 
4. the partitions of data that each worker should train

In [2]:
workers = 3
cpus = 5
job_name = 'worker'
data_split = [
    [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],                # worker 1
    [15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],         # worker 2
    [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]    # worker 3
]

In [3]:
import sys,os,os.path
import urllib3, requests, json
url = os.environ['RUNTIME_ENV_APSX_URL']
token = os.environ['USER_ACCESS_TOKEN']
project_id = os.environ['PROJECT_ID']

function to get internal job id by token, project_id and job_name

In [4]:
def get_job_id(token, project_id, job_name):
    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}
    response = requests.get(url+'/v2/jobs/?project_id=' + project_id,  headers=header)
    response_json = json.loads(response.text)
    results = response_json['results']
    for job in results:
        if job['metadata']['name'] == job_name:
            return job['metadata']['asset_id']
    return "none"

function to start job

In [5]:
def start_job(token, project_id, job_id, cpus,data_split):
    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + token}
    payload = {"job_run": {"configuration": {"env_variables": ["cpus="+cpus,"data_split="+data_split],}}}
    response = requests.post(url+'/v2/jobs/'+job_id+'/runs?project_id='+project_id, json=payload, headers=header)
    

kick off jobs on multiple workers

In [6]:
for i in range(workers):
    data = data_split[i]
    start_job(token,project_id,get_job_id(token,project_id,job_name),str(cpus),str(data))

In [7]:
!ls /project_data/data_asset/

keras_mnist_model-0-loss:2.30%-accuracy19.38%-on-4-5.h5
keras_mnist_model-11-loss:2.30%-accuracy20.40%-on-2-5.h5
keras_mnist_model-1-loss:2.30%-accuracy14.09%-on-0-5.h5
keras_mnist_model-2-loss:2.30%-accuracy11.35%-on-2-5.h5
keras_mnist_model-3-loss:2.30%-accuracy11.35%-on-1-5.h5
keras_mnist_model-4-loss:2.30%-accuracy21.92%-on-3-5.h5
keras_mnist_model-6-loss:2.30%-accuracy30.48%-on-2-5.h5
keras_mnist_model-7-loss:2.30%-accuracy16.43%-on-4-5.h5
keras_mnist_model-8-loss:2.30%-accuracy18.29%-on-0-5.h5
keras_mnist_model-9-loss:2.30%-accuracy11.43%-on-1-5.h5
split.csv
